In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torchmetrics import Accuracy, F1Score
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu_device = torch.device("cpu")


%reload_ext autoreload
%autoreload 2

/home/karm/miniconda3/envs/torch310/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### load bert model

In [5]:
from dataset.SST2.dataloader import BertDataset, BertCustomDataset
from models.bert import BERT
MAX_SENT_LENGTH = 56
BATCH_SIZE = 32

In [6]:
which = "base"
bert_model_name = f"bert-{which}-uncased"
bert_model_name

'bert-base-uncased'

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained(bert_model_name)
sst2_model = BERT(n_classes=2, d_model=768, H=50).to(device)

In [10]:
PATH = f"ckpts/{bert_model_name}_SST2_3_2e-05_FULL.pt"
sst2_model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

### Testing

In [11]:
# test dataset
test_dataset= BertDataset(tokenizer, max_length=MAX_SENT_LENGTH, split="test")
test_dataloader=DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE)

In [12]:
# test dataset
model = sst2_model

model.eval()
loop=tqdm(enumerate(test_dataloader),leave=False,total=len(test_dataloader))
total_matches = 0
with torch.no_grad():
    for batch, dl in loop:
        ids, token_type_ids, mask, label = dl['ids'], dl['token_type_ids'], dl['mask'], dl['target']
        output=F.softmax(model(ids=ids,mask=mask,token_type_ids=token_type_ids), dim=1)
        pred = torch.argmax(output, dim=-1)
        matches = torch.sum(pred == label)
        total_matches += matches.item()


    print(f"Test Accuracy :{total_matches/len(test_dataset)}")  

  0%|                                                              | 0/28 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/karm/miniconda3/envs/torch310/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
 

Test Accuracy :0.9139908256880734


### Custom sentences

In [13]:
def get_predictions_sst2(model, tokenizer, custom_sentences):
    custom_dataset = BertCustomDataset(tokenizer, 56, custom_sentences, device=device)
    custom_dataloader = DataLoader(custom_dataset, batch_size=len(custom_sentences))
    model.eval()
    
    with torch.no_grad():    
        data = next(iter(custom_dataloader))
        ids, token_type_ids, mask = data['ids'], data['token_type_ids'], data['mask']
        output=F.softmax(model(ids=ids,mask=mask,token_type_ids=token_type_ids), dim=1)


        df = pd.DataFrame(custom_sentences, columns=["input"])
        df["Prediction"] = torch.argmax(output, dim=-1).to(cpu_device)
        df["Prediction"] = df["Prediction"].apply(lambda x: {0: "Negative", 1: "Positive"}[x])
        df["Probability"] = torch.max(output, dim=-1).values.to(cpu_device)
    return df

In [14]:
custom_sentences = ["You made 100 run in only 50 balls", 
                    "fantastic! You made only 1 run in 50 balls", 
                    "Despite being a topper you are just passed", 
                    "you have not studied, still you are passed, great!"]
get_predictions_sst2(sst2_model, tokenizer, custom_sentences)

,input,Prediction,Probability
0,You made 100 run in only 50 balls,Negative,0.947929
1,fantastic! You made only 1 run in 50 balls,Positive,0.833501
2,Despite being a topper you are just passed,Negative,0.688427
3,"you have not studied, still you are passed, great!",Positive,0.809652
